In [3]:
# Moduels 1
import os
import pymysql
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import missingno as msno
%matplotlib inline 
import warnings
import matplotlib.font_manager as fm
plt.rcParams['font.family'] = 'NanumGothic'
import warnings
warnings.filterwarnings('ignore')
import math

# Moduels 2
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K

In [4]:
# Connect 
def connect():##db명 String으로 줘야함
  conn = pymysql.connect(
    host='10.125.121.177', 
    user='musthave',
    password="tiger", db="bigdata", charset='utf8')
  if(conn.open):
    print("MySQL 서버에 연결 성공하였습니다.",conn.user,conn.db)
  else:
    print("연결 실패")
  return conn

## Test
connect() # 성공확인

MySQL 서버에 연결 성공하였습니다. b'musthave' b'bigdata'


In [5]:
def getAllData():
  conn = connect()
  cursor = conn.cursor(pymysql.cursors.SSDictCursor)
  # sql = "select * from MyTest"
  sql = "select * from rawdata"
  cursor.execute(query=sql)
  result = cursor.fetchall()
  conn.close()
  # for row in cursor:
  #   print(row)
  return result

def insertDBList(lst): ## M,C,avg,std
  conn = connect() 
  cursor = conn.cursor()
  for i in range(1,len(lst)):
    sql = "insert into Usableitems("+"Machinery,Citem,avg,std)"+" VALUES (%s,%s,%s,%s)"
    cursor.execute(sql,lst[i])
    conn.commit()
  conn.close()
  return True

templist = getAllData()
# print(list)
# print(templist[0].keys())
###

MySQL 서버에 연결 성공하였습니다. b'musthave' b'bigdata'


In [6]:
useKey = ['Machinery','청구품목'] 
MC = []
CI = []
# PNO = []
for i in templist:
  if int(i['리드타임_음수제거'])<=500:
    MC.append(i[useKey[0]].strip())
    CI.append(i[useKey[1]].strip())
    # PNO.append(i[useKey[2]].strip())

MC = list(set(MC))
CI = list(set(CI))
# PNO = list(set(PNO))
## 데이터 내에 양쪽 공백 제거했음.

In [7]:
print(len(MC))
print(len(CI))
# print(len(PNO))


134
3737


In [8]:
# MC, CI, PNO 만들기
mainList = []

# 시간복잡도 : O(n^3) -> O(n^2) 
# 연산횟수 : 2,725,625,794 -> 500758
# 많긴 한데 어쩔수없음. => 실행시간 
for i in MC:
  for j in CI:
    temp = [i,j]
    mainList.append(temp)
    ## mainList :: [mc,ci] 로 구성된 배열
  
for i in mainList:
  i.append([])
# print(mainList)
len(mainList)
## mainList :: [mc,ci,[~~~~~]]

500758

In [10]:
resultMain = []
for i in mainList:
  for j in templist:
    if j['Machinery'].strip() == i[0] and j['청구품목'].strip() == i[1]:
      i[2].append(j['리드타임_음수제거'])

realMainList=[]
for i in mainList:
  if len(i[1])!=0:
    realMainList.append(i)

resultList = []
for i in realMainList:
  one = i[0]
  two = i[1]
  three = np.mean(i[2])
  fore = np.std(i[2])
  resultList.append([one,two,three,fore])
resultList.insert(0,['Machinery','Citem','avg','std'])
# print(resultList)
useList = []
for i in resultList:
  i = tuple(i)
  useList.append(i)
print(useList)
## resultList ==> [(),(),...] 이런 형태

# print(resultList)
# insertDBList(resultList)

In [ ]:
insertDBList(resultList)

MySQL 서버에 연결 성공하였습니다. b'slows14tem' b'bigdata'


True